* 들어가기 전

■ Ensemble?

-> 여러개의 모델(약분류기)을 결합하여 이용하여 보다 더 나은 성능의 모델(강분류기)을 만든다(정확성↑). 배깅과 부스팅이 있다.


※Voting

-> 방식에 따라 hard, soft로 나뉜다.


1) Hard voting
-> mode. 가장 많이 도출된 분류 결과를 최종 분류결과로 결정한다.


2) Soft voting
-> 각 label의 예측률의 mean으로 최종 분류를 한다.



※ classifier의 조건 :

1) 각 classifier는 (1/범주수) 이상의 정확도를 가져야 함.

2) 각 classifier는 상호 독립.

3) 총 classifier의 수에는 제한이 없다.(무한대)









■ Bagging(Bootstrap AGGreration) - homogenous

: Bootstrap한 모델들을 각각 학습시켜 최종 결과물을 합하여(aggregate) 집계.

  (집계방식 : Categorial형은 voting, Continuous는 mean)


 ＊Bootstrap?
    
 -> 표본에서 다시 '복원'표본 추출, 각 표본의 통계량을 계산.


![title](C:\Users\hanri\Downloads/file_1.png)



=> 모델의 bias를 유지하며 var을 줄이기 위함.(overfit 방지), 학습되지 않은 데이터는 error를 뽑아 OOB error(out of bag error)값을 구한다.


In [8]:
from sklearn.ensemble import BaggingClassifier


base_estimator : 임의의 subset data에 적용시킬 기본 모델, 별도의 지정이 없을 경우 Decision Tree를 기본으로 사용한다.

n_estimators : 생성할 base_estimator의 개수를 정한다.

max_samples : base_estimator에 fit시킬 sample data의 개수 [int - 개수 / float - X의 비율 (퍼센트로 적용)]

bootstrap : sample data 생성 시 복원 추출 여부 (Boolean :True - bagging / False - pasting)

In [9]:
bag = BaggingClassifier(n_estimators = 100, bootstrap = True) # Bagging

§ex1) Random Forest(deep tree에 초점)


: 랜덤하게 변수 선택하여 중요도 높은 순대로 결정(모든 변수 선택x - 동일 결과 트리 방지).


![title](C:\Users\hanri\Downloads/image_1.jpg)

In [3]:
from sklearn.ensemble import RandomForestClassifier


n_estimators  = 하위 모델 갯수

max_features  = 0~1, 최대 feature수(default는 sqrt(features))

max_depth  = 트리의 최대 깊이(얼마나 자세하게?)

max_leaf_nodes = 마지막 노드 갯수

min_samples_split  = 각 노드 분기시점에서의 최소 샘플수

criterion = split 방법

In [5]:
RF = RandomForestClassifier(criterion = 'entropy', n_estimators = 15,
                           random_state = 2, max_features = 0.3, max_depth = 2,
                           max_leaf_nodes = 2)
# Random Forest modeling

§ex2) Extra trees

: 랜덤 포레스트와는 달리 후보의 특성을 활용해 분할(최적 threshold를 찾지않아 시간 단축 가능)

=> Random Forest과는 달리 outliers 영향↓


In [6]:
from sklearn.ensemble import ExtraTreesClassifier


n_estimators : 생성할 트리의 개수

criterion : { 'gini', 'entropy' } 등, split을 판단할 기준 설정

max_depth : 각 트리의 최대 깊이 - 미설정시 모든 리프 노드가 pure할때까지 or 
min_samples_split개 이하의 sample을 보유하고 있을 때까지 분류

min_samples_split : split할 수 있는 최저한의 sample 수 설정, ( default=2 )

In [7]:
ET = ExtraTreesClassifier(n_estimators = 50, random_state = 1,
                         criterion = 'gini', max_depth = 2)
# Extra trees modeling

■ Boosting - homogenous

: weight를 활용, 모델 간 연결고리가 있다.
(그 전 모델에서 분류 잘못한 값을 가중치를 두어 그 다음 모델값에 넣는다. 다음 단계 진행시 전 모델의 weight와 선형결합하여 분류하고 오분류된 데이터를 다시 가중치 적용해 그 다음 값에 넣고 반복한다.)

* weight : 오분류(weak)에 높은 가중치, 정분류된 곳에 낮은 가중치(오답에 집중)

=> 여러 모델을 consequent하게 학습, bias를 줄이기 위함.(weak한 모델을 strong하게 만드는데 초점)

![title](C:\Users\hanri\Downloads/image_3.jpg)

§ex1) Adaptive boosting( = adaboost) 

-> 이전 분류기가 틀린 부분을 adaptive하게 바꿔가며 local optimisation (제일 weak한 모델 찾기에 중점)

※ 과정 : 

1) weak model으로 우선 분류(결과 좋을수록 발언권↑)

2) bootstraping하여 가중 샘플링, model이 새로 학습되어 error를 개선, 반복


![title](C:\Users\hanri\Downloads/image_2.jpg)

In [10]:
from sklearn.ensemble import AdaBoostClassifier


n_estimators = 모델 수

learning_rate = 모델 가중치

base_estimator = weak model을 학습할 알고리즘

In [11]:
ada = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1,
                         random_state=0)
# Adaboost modeling

§ex2) Gradient boosting(GB)

-> 경사(gradient)를 이용해 최소의 loss function을 구하고 오류(pseudo-residuals)를 정량화 한다.

※ 과정 :

1) possible weak model을 pseudo-residuals에 fit 한다.

2) 최적의 step size(시행 횟수)값 도출

3) 해당 step size로 앙상블 모델을 업데이트 해나간다.

4) 새로운 pseudo-residuals 도출 - 반복.


-> 상이한 임의 loss func에 대한 adaboost의 일반화형이다.

※ 1) XGboost(level wise)

-> gradient 알고리즘을 분산환경에서 실행할 수 있도록 구현. 병렬처리하며 end-to-end tree boosting. 모든 시나리오에 대한 확장성이 높다.

   파이썬에서 사용시 범주형 변수를 숫자형으로 바꿔야 함(대체로 one-hot encoding을 사용하나, high cardinality features에서는 불안정하다. 따라서 범주형을 2개의 subset으로 나누는 방법을 이용한다)
   
 * 특징 : CART기반(classification, reg tree기반, 조기 종료 가능)
 
 ![title](C:\Users\hanri\Downloads/image_4.jpg)
 

In [4]:
import xgboost as xgb
# 따로 제공해주지 않아 cmd에서 install필요.

ModuleNotFoundError: No module named 'xgboost'

Learning_rate = 0~1사이값, step size

max_depth = 얼마나 깊은 트리를 만들지

subsample = 트리 당 features의 퍼센트(낮은 값은 underfit 발생)

n_estimators = 트리 수

objective = loss functiion

gamma = expected reduction에 기반해 split?

alpha = L1 regularization on leaf weights

lambda = L2 regulatization on leaf weights(L1보다 smooth)

In [1]:
xg = xgb.XGBRegressor(objective = 'reg:linear', n_estimators = 10)

NameError: name 'xgb' is not defined

※ 2) LightGBM(LGBM) - Leaf wise

-> xgboost와 비교해서 비교적 빠르며, high cardinality features처리시 유용.
   feature를 각 step에서 통계량으로 전환-기존 효율성의 trade-off해결.
   
   but, 고차원 데이터는 처리하기 힘들다.

In [ ]:
import lightgbm as LGBMClassifier


xgboost와 상이,

num_leaves = 하나의 트리가 가지는 최대 leaf

min_child_samples = leaf node를 위한 최소한의 sample data 수

feature_fraction = feature 비율

In [ ]:
lgb = LGBMClassifier(n_estimators = 400)

※ 3) Catboost(level wise)

-> categorial 변수를 처리하는데 유용한 알고리즘. ordering -principle 개념을 대입하여 data leakage에 의한 문제, high cardinality 문제 해결.

* ordering-principle - 자연수 정렬성

: S의 모든 원소 s에 대하여 m<=s을 만족하는 m이 존재한다.(최소항이 존재)

In [6]:
from catboost import CatBoostClassifier
# cmd에서 설치 필요

ModuleNotFoundError: No module named 'catboost'

iteration = 반복 수

learning_rate = 0~1, weight

loss_function = loss func 지정

In [ ]:
clf = CatBoostClassifier(iterations = 10, verbose = 5)

■ Bagging, Boosting의 장단점, 차이

-> 부스팅이 에러율이 적고(성능↑), 하지만 느리며 overfit 위험 o. 노이즈/outliers 의 위험도 있다

   배깅은 노이즈에 강하며 overfit방지 가능. 그러나 느리며 모형 해석 능력x(시각화 불가)
   
   
   shallow한 tree는 var은 낮으나 bias가 높다. 반면 deep한 tree는 그 반대이다.
   
   bagging은 병렬학습, 독립적인 반면 boosting은 순차학습. 서로 연관되어 있다.
   
   ![title](C:\Users\hanri\Downloads/image_5.jpg)

■ Stacking - heterogeneous

: 서로 다른 모델을 융합, meta model 활용.

※ 과정 :

1) 데이터를 나눈다

2) 첫 fold에 weak model을 학습

3) 그 모델에 2nd fold를 넣어 predict

4) 해당 결과물을 meta model의 input으로 학습


![title](C:\Users\hanri\Downloads/image_6.jpg)

In [13]:
from sklearn.ensemble import StackingClassifier


final_estimator = 마지막에 모델을 합칠 classifier

stack_method = base estimator. 이 파라미터로 stacking 

passthrough = False시 final estimator는 예측에만 사용된다


In [16]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier


estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           LinearSVC(random_state=42)))]

clf = StackingClassifier(
     estimators = estimators, final_estimator = LogisticRegression())

# Stacking ensemble modeling

참고 : https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205?gi=7bc88250d3f5